### **Summarization: An Unsupervised Extractive Way**

###### **There are two types of approach for text summarization: 1) Supervised and 2) Unsupervised. And in terms of summaries, there are two types of summaries: 1) Extractive and 2) Abstractive.<br>Here, we will use a Unsupervised extractive method.**

#### **Steps**

##### **Import Libraries**

In [3]:
import nltk
import re
import string
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize as nlkt_sent_tokenize
from nltk.tokenize import word_tokenize as nlkt_word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords
import numpy as np
from scipy.spatial.distance import cosine

##### **1. Generate Embedding Model**
###### **Embedding selects sentences that contain words with the same meaning as the most relevant words — even if the words are different. So, before even preprocessing the text, the program must define the embedding model. To obtain the word embedding, I split the text into words and passed it to Word2vec.** 

In [5]:
# Define Clean_SEntence Module
def clean_sentences(text):
    stop_words = set(stopwords.words('english'))
    sentences = sent_tokenize(text)
    sentences_cleaned = []
    for sent in sentences:
        words = nlkt_word_tokenize(sent)
        words = [w for w in words if w not in string.punctuation]
        words = [w for w in words if not w.lower() in stop_words]
        words = [w.lower() for w in words]
        sentences_cleaned.append(" ".join(words))
    return sentences_cleaned

In [6]:
# Define Embedding Model
for sent in clean_sentences:
    words.append(nlkt_word_tokenize(sent))
model = Word2Vec(words, min_count=1, sg = 1)

NameError: name 'clean_sentences' is not defined